In [ ]:
# Modelagem Machine Learning - Employee Attrition
# Comparação de algoritmos e otimização

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (classification_report, confusion_matrix, roc_auc_score, 
                           roc_curve, precision_recall_curve, accuracy_score, 
                           precision_score, recall_score, f1_score)
import xgboost as xgb
import shap
import warnings
warnings.filterwarnings('ignore')

class AttritionModelingPipeline:
    """Pipeline completo para modelagem de Employee Attrition"""
    
    def __init__(self, random_state=42):
        self.random_state = random_state
        self.models = {}
        self.best_models = {}
        self.results = {}
        self.cv_scores = {}
        
    def load_processed_data(self, data_dir='processed_data/'):
        """Carrega dados pré-processados"""
        
        print("="*60)
        print("🤖 MODELAGEM MACHINE LEARNING - EMPLOYEE ATTRITION")
        print("="*60)
        
        try:
            self.X_train = pd.read_csv(f'{data_dir}X_train.csv')
            self.X_test = pd.read_csv(f'{data_dir}X_test.csv')
            self.y_train = pd.read_csv(f'{data_dir}y_train.csv').squeeze()
            self.y_test = pd.read_csv(f'{data_dir}y_test.csv').squeeze()
            
            # Tentar carregar validation se existir
            try:
                self.X_val = pd.read_csv(f'{data_dir}X_val.csv')
                self.y_val = pd.read_csv(f'{data_dir}y_val.csv').squeeze()
                print(f"✅ Validation set carregado: {self.X_val.shape}")
            except:
                self.X_val = None
                self.y_val = None
                print("ℹ️ Validation set não encontrado")
            
            print(f"✅ Dados carregados:")
            print(f"  Train: {self.X_train.shape}")
            print(f"  Test: {self.X_test.shape}")
            print(f"  Features: {self.X_train.shape[1]}")
            
        except Exception as e:
            print(f"❌ Erro ao carregar dados: {e}")
            print("💡 Execute primeiro o pré-processamento!")
            return False
            
        return True
    
    def initialize_models(self):
        """Inicializa diferentes modelos para comparação"""
        
        print(f"\n🔧 INICIALIZANDO MODELOS:")
        
        self.models = {
            'Logistic Regression': LogisticRegression(random_state=self.random_state, max_iter=1000),
            'Random Forest': RandomForestClassifier(random_state=self.random_state, n_estimators=100),
            'XGBoost': xgb.XGBClassifier(random_state=self.random_state, eval_metric='logloss'),
            'SVM': SVC(random_state=self.random_state, probability=True),
            'Gradient Boosting': GradientBoostingClassifier(random_state=self.random_state),
            'KNN': KNeighborsClassifier(n_neighbors=5),
            'Naive Bayes': GaussianNB()
        }
        
        print(f"✅ {len(self.models)} modelos inicializados:")
        for name in self.models.keys():
            print(f"  - {name}")
    
    def evaluate_baseline_models(self, cv_folds=5):
        """Avalia modelos baseline com cross-validation"""
        
        print(f"\n📊 AVALIAÇÃO DE MODELOS BASELINE (CV={cv_folds}):")
        print("-" * 70)
        
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=self.random_state)
        
        metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
        self.cv_scores = {metric: {} for metric in metrics}
        
        for name, model in self.models.items():
            print(f"\n🔄 Avaliando {name}...")
            
            # Cross-validation para cada métrica
            for metric in metrics:
                scores = cross_val_score(model, self.X_train, self.y_train, 
                                       cv=cv, scoring=metric, n_jobs=-1)
                self.cv_scores[metric][name] = scores
                
                print(f"  {metric.upper()}: {scores.mean():.3f} (+/- {scores.std() * 2:.3f})")
        
        # Criar DataFrame com resultados
        results_list = []
        for name in self.models.keys():
            row = {'Model': name}
            for metric in metrics:
                scores = self.cv_scores[metric][name]
                row[f'{metric.capitalize()}_Mean'] = scores.mean()
                row[f'{metric.capitalize()}_Std'] = scores.std()
            results_list.append(row)
        
        self.baseline_results = pd.DataFrame(results_list)
        
        print(f"\n📈 RANKING DOS MODELOS (por AUC-ROC):")
        ranking = self.baseline_results.sort_values('Roc_auc_Mean', ascending=False)
        print(ranking[['Model', 'Accuracy_Mean', 'Precision_Mean', 'Recall_Mean', 'F1_Mean', 'Roc_auc_Mean']].round(3))
        
        return self.baseline_results
    
    def hyperparameter_tuning(self, top_models=3):
        """Otimização de hiperparâmetros para os melhores modelos"""
        
        print(f"\n🎯 OTIMIZAÇÃO DE HIPERPARÂMETROS:")
        
        # Selecionar top modelos
        top_model_names = (self.baseline_results
                          .sort_values('Roc_auc_Mean', ascending=False)
                          .head(top_models)['Model'].tolist())
        
        print(f"Otimizando top {top_models} modelos: {top_model_names}")
        
        # Definir grids de hiperparâmetros
        param_grids = {
            'Logistic Regression': {
                'C': [0.1, 1, 10, 100],
                'penalty': ['l1', 'l2'],
                'solver': ['liblinear']
            },
            'Random Forest': {
                'n_estimators': [100, 200, 300],
                'max_depth': [10, 20, None],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            },
            'XGBoost': {
                'n_estimators': [100, 200, 300],
                'max_depth': [3, 6, 9],
                'learning_rate': [0.01, 0.1, 0.2],
                'subsample': [0.8, 0.9, 1.0]
            },
            'SVM': {
                'C': [0.1, 1, 10, 100],
                'gamma': ['scale', 'auto', 0.001, 0.01],
                'kernel': ['rbf', 'poly']
            },
            'Gradient Boosting': {
                'n_estimators': [100, 200],
                'max_depth': [3, 5, 7],
                'learning_rate': [0.01, 0.1, 0.2]
            }
        }
        
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=self.random_state)
        
        for model_name in top_model_names:
            if model_name in param_grids:
                print(f"\n🔍 Otimizando {model_name}...")
                
                model = self.models[model_name]
                param_grid = param_grids[model_name]
                
                # Grid Search
                grid_search = GridSearchCV(
                    model, param_grid, cv=cv, scoring='roc_auc', 
                    n_jobs=-1, verbose=0
                )
                
                grid_search.fit(self.X_train, self.y_train)
                
                self.best_models[model_name] = grid_search.best_estimator_
                
                print(f"  ✅ Melhores parâmetros: {grid_search.best_params_}")
                print(f"  ✅ Melhor score: {grid_search.best_score_:.3f}")
            else:
                # Para modelos sem grid, usar modelo padrão
                self.best_models[model_name] = self.models[model_name]
        
        print(f"\n🎉 Otimização concluída para {len(self.best_models)} modelos")
    
    def evaluate_final_models(self):
        """Avalia modelos otimizados no conjunto de teste"""
        
        print(f"\n🏆 AVALIAÇÃO FINAL DOS MODELOS:")
        print("-" * 60)
        
        self.final_results = {}
        
        for name, model in self.best_models.items():
            print(f"\n📊 Avaliando {name}...")
            
            # Treinar modelo
            model.fit(self.X_train, self.y_train)
            
            # Predições
            y_pred = model.predict(self.X_test)
            y_pred_proba = model.predict_proba(self.X_test)[:, 1]
            
            # Calcular métricas
            metrics = {
                'accuracy': accuracy_score(self.y_test, y_pred),
                'precision': precision_score(self.y_test, y_pred),
                'recall': recall_score(self.y_test, y_pred),
                'f1': f1_score(self.y_test, y_pred),
                'roc_auc': roc_auc_score(self.y_test, y_pred_proba)
            }
            
            self.final_results[name] = {
                'model': model,
                'predictions': y_pred,
                'probabilities': y_pred_proba,
                'metrics': metrics
            }
            
            # Mostrar métricas
            for metric, value in metrics.items():
                print(f"  {metric.upper()}: {value:.3f}")
        
        # Criar DataFrame com resultados finais
        final_df = pd.DataFrame({
            name: result['metrics'] for name, result in self.final_results.items()
        }).T
        
        print(f"\n🥇 RANKING FINAL DOS MODELOS:")
        print(final_df.sort_values('roc_auc', ascending=False).round(3))
        
        return final_df
    
    def create_model_comparison_plots(self):
        """Cria visualizações comparativas dos modelos"""
        
        print(f"\n📊 CRIANDO VISUALIZAÇÕES COMPARATIVAS:")
        
        fig, axes = plt.subplots(2, 3, figsize=(20, 12))
        fig.suptitle('🤖 COMPARAÇÃO DE MODELOS - EMPLOYEE ATTRITION', fontsize=20, fontweight='bold')
        
        # 1. Comparação de métricas
        ax1 = axes[0, 0]
        metrics_df = pd.DataFrame({
            name: result['metrics'] for name, result in self.final_results.items()
        }).T
        metrics_df[['accuracy', 'precision', 'recall', 'f1']].plot(kind='bar', ax=ax1, alpha=0.8)
        ax1.set_title('Métricas por Modelo', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Score')
        ax1.tick_params(axis='x', rotation=45)
        ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax1.grid(axis='y', alpha=0.3)
        
        # 2. ROC Curves
        ax2 = axes[0, 1]
        colors = plt.cm.Set1(np.linspace(0, 1, len(self.final_results)))
        
        for (name, result), color in zip(self.final_results.items(), colors):
            fpr, tpr, _ = roc_curve(self.y_test, result['probabilities'])
            auc_score = result['metrics']['roc_auc']
            ax2.plot(fpr, tpr, color=color, label=f'{name} (AUC = {auc_score:.3f})', linewidth=2)
        
        ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5)
        ax2.set_xlabel('False Positive Rate')
        ax2.set_ylabel('True Positive Rate')
        ax2.set_title('ROC Curves Comparison', fontsize=14, fontweight='bold')
        ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax2.grid(alpha=0.3)
        
        # 3. Precision-Recall Curves
        ax3 = axes[0, 2]
        
        for (name, result), color in zip(self.final_results.items(), colors):
            precision, recall, _ = precision_recall_curve(self.y_test, result['probabilities'])
            ax3.plot(recall, precision, color=color, label=name, linewidth=2)
        
        ax3.set_xlabel('Recall')
        ax3.set_ylabel('Precision')
        ax3.set_title('Precision-Recall Curves', fontsize=14, fontweight='bold')
        ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        ax3.grid(alpha=0.3)
        
        # 4. Matriz de Confusão do Melhor Modelo
        best_model_name = metrics_df.sort_values('roc_auc', ascending=False).index[0]
        best_predictions = self.final_results[best_model_name]['predictions']
        
        ax4 = axes[1, 0]
        cm = confusion_matrix(self.y_test, best_predictions)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4)
        ax4.set_title(f'Confusion Matrix - {best_model_name}', fontsize=14, fontweight='bold')
        ax4.set_ylabel('Actual')
        ax4.set_xlabel('Predicted')
        
        # 5. Distribuição de Probabilidades
        ax5 = axes[1, 1]
        
        best_probabilities = self.final_results[best_model_name]['probabilities']
        
        # Separar por classe real
        prob_class_0 = best_probabilities[self.y_test == 0]
        prob_class_1 = best_probabilities[self.y_test == 1]
        
        ax5.hist(prob_class_0, bins=30, alpha=0.7, label='No Attrition', color='blue', density=True)
        ax5.hist(prob_class_1, bins=30, alpha=0.7, label='Attrition', color='red', density=True)
        ax5.set_xlabel('Predicted Probability')
        ax5.set_ylabel('Density')
        ax5.set_title(f'Probability Distribution - {best_model_name}', fontsize=14, fontweight='bold')
        ax5.legend()
        ax5.grid(alpha=0.3)
        
        # 6. Cross-Validation Scores
        ax6 = axes[1, 2]
        
        if hasattr(self, 'cv_scores'):
            cv_auc_scores = pd.DataFrame(self.cv_scores['roc_auc'])
            cv_auc_scores.boxplot(ax=ax6)
            ax6.set_title('Cross-Validation AUC Scores', fontsize=14, fontweight='bold')
            ax6.set_ylabel('AUC Score')
            ax6.tick_params(axis='x', rotation=45)
            ax6.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    def feature_importance_analysis(self):
        """Análise de importância das features"""
        
        print(f"\n🎯 ANÁLISE DE IMPORTÂNCIA DAS FEATURES:")
        
        # Obter melhor modelo
        metrics_df = pd.DataFrame({
            name: result['metrics'] for name, result in self.final_results.items()
        }).T
        best_model_name = metrics_df.sort_values('roc_auc', ascending=False).index[0]
        best_model = self.final_results[best_model_name]['model']
        
        print(f"Analisando importâncias do melhor modelo: {best_model_name}")
        
        # Feature importance (para modelos tree-based)
        if hasattr(best_model, 'feature_importances_'):
            importances = best_model.feature_importances_
            feature_names = self.X_train.columns
            
            # Criar DataFrame
            importance_df = pd.DataFrame({
                'Feature': feature_names,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
            
            # Visualizar top 15
            plt.figure(figsize=(12, 8))
            top_features = importance_df.head(15)
            
            plt.barh(range(len(top_features)), top_features['Importance'], color='skyblue')
            plt.yticks(range(len(top_features)), top_features['Feature'])
            plt.xlabel('Feature Importance')
            plt.title(f'🏆 TOP 15 FEATURES MAIS IMPORTANTES - {best_model_name}', 
                     fontsize=16, fontweight='bold')
            plt.gca().invert_yaxis()
            plt.grid(axis='x', alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            print("📊 Top 10 Features Mais Importantes:")
            for i, (_, row) in enumerate(importance_df.head(10).iterrows(), 1):
                print(f"{i:2d}. {row['Feature']}: {row['Importance']:.4f}")
        
        # SHAP Analysis (se disponível)
        try:
            print(f"\n🔍 Iniciando análise SHAP...")
            
            # Criar explainer SHAP
            if best_model_name in ['Random Forest', 'XGBoost', 'Gradient Boosting']:
                explainer = shap.TreeExplainer(best_model)
                shap_values = explainer.shap_values(self.X_test[:100])  # Amostra para velocidade
                
                # Se shap_values for lista (classificação binária), pegar valores da classe 1
                if isinstance(shap_values, list):
                    shap_values = shap_values[1]
                
                # Summary plot
                plt.figure(figsize=(12, 8))
                shap.summary_plot(shap_values, self.X_test[:100], plot_type="bar", show=False)
                plt.title(f'🎯 SHAP Feature Importance - {best_model_name}', 
                         fontsize=16, fontweight='bold', pad=20)
                plt.tight_layout()
                plt.show()
                
                print("✅ Análise SHAP concluída!")
                
        except Exception as e:
            print(f"⚠️ Não foi possível executar análise SHAP: {e}")
    
    def generate_business_insights(self):
        """Gera insights de negócio baseados nos modelos"""
        
        print(f"\n💼 INSIGHTS DE NEGÓCIO:")
        print("="*50)
        
        # Obter melhor modelo e suas predições
        metrics_df = pd.DataFrame({
            name: result['metrics'] for name, result in self.final_results.items()
        }).T
        best_model_name = metrics_df.sort_values('roc_auc', ascending=False).index[0]
        best_result = self.final_results[best_model_name]
        
        # Estatísticas do melhor modelo
        best_metrics = best_result['metrics']
        probabilities = best_result['probabilities']
        predictions = best_result['predictions']
        
        print(f"🏆 MELHOR MODELO: {best_model_name}")
        print(f"   - AUC-ROC: {best_metrics['roc_auc']:.3f}")
        print(f"   - Precisão: {best_metrics['precision']:.3f}")
        print(f"   - Recall: {best_metrics['recall']:.3f}")
        print(f"   - F1-Score: {best_metrics['f1']:.3f}")
        
        # Análise de segmentos de risco
        print(f"\n🎯 SEGMENTAÇÃO DE RISCO:")
        
        # Definir thresholds de risco
        high_risk_threshold = 0.7
        medium_risk_threshold = 0.3
        
        high_risk_count = (probabilities > high_risk_threshold).sum()
        medium_risk_count = ((probabilities > medium_risk_threshold) & 
                           (probabilities <= high_risk_threshold)).sum()
        low_risk_count = (probabilities <= medium_risk_threshold).sum()
        
        total_employees = len(probabilities)
        
        print(f"   - Alto Risco (>{high_risk_threshold:.1%}): {high_risk_count} funcionários ({high_risk_count/total_employees:.1%})")
        print(f"   - Médio Risco ({medium_risk_threshold:.1%}-{high_risk_threshold:.1%}): {medium_risk_count} funcionários ({medium_risk_count/total_employees:.1%})")
        print(f"   - Baixo Risco (<{medium_risk_threshold:.1%}): {low_risk_count} funcionários ({low_risk_count/total_employees:.1%})")
        
        # ROI potencial
        print(f"\n💰 ESTIMATIVA DE ROI:")
        
        # Assumindo custos típicos de RH
        cost_replacement = 20000  # Custo médio de substituição
        cost_retention_program = 2000  # Custo de programa de retenção
        
        # Funcionários que realmente saíram e foram identificados corretamente
        true_positives = ((self.y_test == 1) & (predictions == 1)).sum()
        
        # Economia potencial se conseguirmos reter 50% dos identificados
        retention_success_rate = 0.5
        potential_savings = true_positives * retention_success_rate * cost_replacement
        program_costs = high_risk_count * cost_retention_program
        net_savings = potential_savings - program_costs
        
        print(f"   - Funcionários identificados corretamente para sair: {true_positives}")
        print(f"   - Economia potencial (50% retenção): ${potential_savings:,.0f}")
        print(f"   - Custos do programa de retenção: ${program_costs:,.0f}")
        print(f"   - Economia líquida estimada: ${net_savings:,.0f}")
        
        if net_savings > 0:
            roi = (net_savings / program_costs) * 100
            print(f"   - ROI estimado: {roi:.1f}%")
        
        # Recomendações específicas
        print(f"\n📋 RECOMENDAÇÕES DE AÇÃO:")
        print("   1. 🚨 PRIORIDADE ALTA - Funcionários com risco > 70%:")
        print("      • Reunião individual imediata com gestor")
        print("      • Revisão salarial e plano de carreira")
        print("      • Programa de mentoria personalizado")
        
        print("   2. ⚠️ PRIORIDADE MÉDIA - Funcionários com risco 30-70%:")
        print("      • Survey de satisfação direcionado")
        print("      • Melhorias no ambiente de trabalho")
        print("      • Programa de desenvolvimento profissional")
        
        print("   3. ✅ MANUTENÇÃO - Funcionários com risco < 30%:")
        print("      • Reconhecimento e feedback positivo")
        print("      • Programa de embaixadores internos")
        print("      • Manutenção das boas práticas atuais")
        
        return {
            'best_model': best_model_name,
            'metrics': best_metrics,
            'risk_segments': {
                'high_risk': high_risk_count,
                'medium_risk': medium_risk_count,
                'low_risk': low_risk_count
            },
            'roi_analysis': {
                'potential_savings': potential_savings,
                'program_costs': program_costs,
                'net_savings': net_savings
            }
        }
    
    def run_complete_modeling_pipeline(self, data_dir='processed_data/'):
        """Executa pipeline completo de modelagem"""
        
        print("🚀 EXECUTANDO PIPELINE COMPLETO DE MODELAGEM")
        print("="*70)
        
        # 1. Carregar dados
        if not self.load_processed_data(data_dir):
            return None
        
        # 2. Inicializar modelos
        self.initialize_models()
        
        # 3. Avaliação baseline
        baseline_results = self.evaluate_baseline_models()
        
        # 4. Otimização de hiperparâmetros
        self.hyperparameter_tuning()
        
        # 5. Avaliação final
        final_results = self.evaluate_final_models()
        
        # 6. Visualizações
        self.create_model_comparison_plots()
        
        # 7. Análise de features
        self.feature_importance_analysis()
        
        # 8. Insights de negócio
        business_insights = self.generate_business_insights()
        
        print("\n🎉 PIPELINE DE MODELAGEM CONCLUÍDO COM SUCESSO!")
        print("="*70)
        
        return {
            'baseline_results': baseline_results,
            'final_results': final_results,
            'best_models': self.best_models,
            'business_insights': business_insights
        }
    
    def save_model_results(self, results, output_dir='model_results/'):
        """Salva resultados dos modelos"""
        
        import os
        import joblib
        
        os.makedirs(output_dir, exist_ok=True)
        
        # Salvar melhor modelo
        best_model_name = list(results['business_insights']['best_model'])
        best_model = self.best_models[best_model_name]
        
        joblib.dump(best_model, f'{output_dir}best_model.pkl')
        
        # Salvar resultados
        results['baseline_results'].to_csv(f'{output_dir}baseline_results.csv', index=False)
        results['final_results'].to_csv(f'{output_dir}final_results.csv')
        
        print(f"💾 Resultados salvos em: {output_dir}")


# ========================================
# EXEMPLO DE USO
# ========================================

if __name__ == "__main__":
    
    # Inicializar pipeline
    modeling_pipeline = AttritionModelingPipeline(random_state=42)
    
    # Executar pipeline completo
    results = modeling_pipeline.run_complete_modeling_pipeline()
    
    if results:
        # Salvar resultados
        modeling_pipeline.save_model_results(results)
        
        print(f"\n📊 RESUMO DOS RESULTADOS:")
        print(f"Melhor modelo: {results['business_insights']['best_model']}")
        
        best_metrics = results['business_insights']['metrics']
        print(f"AUC-ROC: {best_metrics['roc_auc']:.3f}")
        print(f"F1-Score: {best_metrics['f1']:.3f}")
        
        roi_analysis = results['business_insights']['roi_analysis']
        print(f"ROI estimado: ${roi_analysis['net_savings']:,.0f}")
        
        print(f"\n🎯 PRÓXIMOS PASSOS:")
        print("1. Criar dashboard interativo")
        print("2. Implementar sistema de monitoramento")
        print("3. Desenvolver API para predições em tempo real")
        print("4. Criar relatórios executivos automatizados")